In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
🧱 1. Postgres Task Memory Table

CREATE TABLE agent_tasks (
  id SERIAL PRIMARY KEY,
  task TEXT,
  result TEXT,
  status TEXT,
  timestamp TIMESTAMP DEFAULT NOW(),
  retry_count INT DEFAULT 0,
  vector_embedding vector(1536)
);




⚙️ 2. LangGraph State + Nodes for Task Planning

from datetime import datetime, timedelta

def create_task_node(state):
    # If no task: generate based on user input
    state['task'] = f"Research about: {state['goal']}"
    return state

def execute_task_node(state):
    # Simulate execution result
    result = f"Finished task: {state['task']} ✅"
    state['result'] = result
    state['status'] = "completed"
    return state

def save_task_memory(state):
    vector_store.add_texts(
        texts=[state['result']],
        metadatas=[{
            "task": state['task'],
            "status": state['status'],
            "timestamp": datetime.utcnow().isoformat()
        }]
    )
    return state




🧠 3. Retry Failed Tasks (Temporal Logic)

def retry_failed_tasks_node(state):
    # ⏱️ Look for tasks older than X minutes + failed
    past_tasks = vector_store.similarity_search("status: failed", k=3)
    retries = []
    for doc in past_tasks:
        if "failed" in doc.metadata["status"]:
            task_time = datetime.fromisoformat(doc.metadata["timestamp"])
            if datetime.utcnow() - task_time > timedelta(minutes=5):
                retries.append(doc.page_content)
    state['retry_tasks'] = retries
    return state




🤖 AGS Baby AGI LangGraph Flow

graph = StateGraph()

graph.add_node("CreateTask", create_task_node)
graph.add_node("ExecuteTask", execute_task_node)
graph.add_node("SaveMemory", save_task_memory)
graph.add_node("RetryFailed", retry_failed_tasks_node)

graph.set_entry_point("RetryFailed")
graph.add_edge("RetryFailed", "CreateTask")
graph.add_edge("CreateTask", "ExecuteTask")
graph.add_edge("ExecuteTask", "SaveMemory")
graph.add_edge("SaveMemory", "RetryFailed")

baby_agi = graph.compile()




🧪 Run Example

baby_agi.invoke({
    "goal": "Summarize AGS reasoning agents",
})




🔄 Loop Behavior

Cycle	Task	Status	Next

1	"Summarize..."	failed	Retry in 5 mins
2	Retry old task	completed	Add memory
3	New goal	in progress	...





🧠 Advantages

🕰️ Time-aware retries and scheduling

🧠 Memory replay to avoid task forgetfulness

📚 Structured DB storage of results

🍼 Mimics BabyAGI: think → act → reflect → retry

SyntaxError: invalid character '🧱' (U+1F9F1) (3682358795.py, line 1)